# Point Source to Far Field Reflector — One-Shot Benchmark

**`Kernel → Restart & Run All`** — runs the full pipeline without any manual steps:

| Step | What happens |
|------|--------------|
| 1 | **Config** — sizes, benchmark case |
| 2 | **Dependencies** — installs numpy + matplotlib into the live kernel |
| 3 | **Generate** point clouds (quasi-Monte Carlo on sphere) |
| 4 | **Compile** C++ via `make` |
| 5 | **Run** the benchmark |
| 6 | **Visualize** — runs in a subprocess so fresh installs are picked up |

## Step 1 — Configuration

In [ ]:
# ── Point-cloud sizes ─────────────────────────────────────────────────────────
# NK complexity is O(NK²):  1600 → fast (~seconds)  |  16488 → full (~minutes)
NK       = 1600
NK_small = 200     # warm-start sampler  (must be < NK)

# ── Benchmark test-case ───────────────────────────────────────────────────────
# 'test_3D_SquareToCircle_logcost_MonteCarlo.h'
# 'test_3D_SquareToTwoGaussSide_logcost_MonteCarlo.h'
BENCHMARK = 'test_3D_SquareToCircle_logcost_MonteCarlo.h'

# ── Paths ─────────────────────────────────────────────────────────────────────
import os, sys
REPO_ROOT = os.path.abspath('')          # directory this notebook lives in
CODE_DIR  = os.path.join(REPO_ROOT, 'BenchmarkCode')

print(f'Python    : {sys.executable}')
print(f'Repo root : {REPO_ROOT}')
print(f'Code dir  : {CODE_DIR}')
print(f'NK={NK}, NK_small={NK_small}')
print(f'Benchmark : {BENCHMARK}')

## Step 2 — Install / fix dependencies

Uses `sys.executable` so pip targets **this exact kernel's** site-packages.  
Matplotlib is force-reinstalled so its C extension matches the numpy version.  
Visualization runs in a **subprocess** later, so the fresh binaries are loaded cleanly.

In [ ]:
import subprocess, sys

def pip(*args):
    """Run pip via the live kernel's interpreter and stream output."""
    cmd = [sys.executable, '-m', 'pip', *args]
    result = subprocess.run(cmd, capture_output=True, text=True)
    # Show only warnings/errors, not the full install log
    for line in (result.stdout + result.stderr).splitlines():
        if any(k in line for k in ('ERROR', 'error', 'WARNING', 'Successfully', 'already')):
            print(line)
    return result.returncode

# Install numpy first so matplotlib's C extension links against the right version
print('Installing numpy...')
pip('install', 'numpy>=1.21,<2.0', '--upgrade', '--quiet')

# Force-reinstall matplotlib so ft2font.so is recompiled / matched to numpy
print('Installing matplotlib (force-reinstall to fix binary compatibility)...')
pip('install', 'matplotlib', '--upgrade', '--force-reinstall', '--quiet')

# Verify
r = subprocess.run(
    [sys.executable, '-c', 'import numpy, matplotlib; print("numpy", numpy.__version__, "| matplotlib", matplotlib.__version__)'],
    capture_output=True, text=True
)
print(r.stdout.strip() or r.stderr.strip())
print('✓ Dependencies ready')

## Step 3 — Generate Point Clouds

In [ ]:
import math

# ── Halton quasi-random sequence ──────────────────────────────────────────────
def halton(index, base):
    result, f, i = 0.0, 1.0, index
    while i > 0:
        f /= base; result += f * (i % base); i //= base
    return result

# ── Inverse stereographic projection → unit sphere ────────────────────────────
# Matches the C++ formula in Generic_3D_logcost_MonteCarlo.h exactly
def sphere_pt(X, Y, upper=True):
    N2 = X*X + Y*Y; d = 1.0 + N2; z = 1.0 if upper else -1.0
    return (2*X/d, 2*Y/d, z*(1-N2)/d)

def gen_pts(n, upper, skip=0, half=0.6):
    pts, idx = [], skip
    while len(pts) < n:
        pts.append(sphere_pt((halton(idx,2)-0.5)*2*half, (halton(idx,3)-0.5)*2*half, upper))
        idx += 1
    return pts

FMT = lambda v: f'{v:.21e}'
row = lambda pt: ', '.join(FMT(v) for v in pt) + ', '

def write_file(path, lines):
    with open(path, 'w', newline='') as f:
        f.write('\r\n'.join(lines) + '\r\n')

# Generate all clouds
print('Generating points...', end=' ', flush=True)
x_pts  = gen_pts(NK,       upper=True,  skip=0)
y_pts  = gen_pts(NK,       upper=False, skip=0)
xs_pts = gen_pts(NK_small, upper=True,  skip=NK)
ys_pts = gen_pts(NK_small, upper=False, skip=NK)
print('done.')

# MonteCarlo_Pointcloud_3D_128.h
write_file(os.path.join(CODE_DIR, 'QuasiMonteCarlo', 'MonteCarlo_Pointcloud_3D_128.h'), [
    f'#ifndef MonteCarlo_Pointcloud_3D_{NK}', f'#define MonteCarlo_Pointcloud_3D_{NK}', '',
    f'const int NK={NK};', 'const int dim=3;', '',
    'double x[NK][dim]=', '{', *[row(p) for p in x_pts], '};', '', '',
    'double y[NK][dim]=', '{', *[row(p) for p in y_pts], '};', '', '',
    f'const int NK_small={NK_small};', '', '',
    'double x_small[NK_small][dim]=', '{', *[row(p) for p in xs_pts], '};', '', '',
    'double y_small[NK_small][dim]=', '{', *[row(p) for p in ys_pts], '};', '', '',
    '#endif',
])

# 3D_MonteCarlo_Pointcloud_small.h
write_file(os.path.join(CODE_DIR, 'SmallGrid', '3D_MonteCarlo_Pointcloud_small.h'), [
    f'const int NK_small={NK_small};', '',
    '//monte-carlo cloud for both x and y. where reflector cost is applied, last values of y should change sign.',
    '', '',
    'double x_small[NK_small][3]=', '{ ', *[row(p) for p in xs_pts], '};',
    '', '', '', '', '', '', '',
    'double y_small[NK_small][3]=', '{ ', *[row(p) for p in ys_pts], '};',
    '', '',
])

# PushForward_Cloud_128.h
write_file(os.path.join(CODE_DIR, 'PushForward', 'PushForward_Cloud_128.h'), [
    '#ifndef PushForward_Cloud_128', '#define PushForward_Cloud_128', '',
    f'const int Push_Cloud_Size={NK};', '',
    'double Push_Cloud[Push_Cloud_Size][3]=', '{', *[row(p) for p in x_pts], '};',
    '', '', '#endif',
])

print(f'✓ Point clouds written: NK={NK}, NK_small={NK_small}')

## Step 4 — Compile

In [ ]:
import re

# Patch the #include in main.cpp to the chosen benchmark
main_cpp = os.path.join(CODE_DIR, 'main.cpp')
src = open(main_cpp).read()
patched = re.sub(
    r'#include\s+"Benchmarks/[^"]+"',
    f'#include "Benchmarks/{BENCHMARK}"',
    src, count=1
)
if patched != src:
    open(main_cpp, 'w').write(patched)
    print(f'Patched main.cpp → {BENCHMARK}')
else:
    print(f'main.cpp already set to {BENCHMARK}')

print('\nCompiling...')
result = subprocess.run(['make', '-C', CODE_DIR], capture_output=True, text=True)
print(result.stdout)
if result.returncode != 0:
    print('STDERR:', result.stderr)
    raise RuntimeError(f'Compilation failed (exit {result.returncode})')
print('✓ Compiled successfully')

## Step 5 — Run Benchmark

In [ ]:
import time

print(f'Running benchmark (NK={NK})...')
t0 = time.time()

result = subprocess.run(
    [os.path.join(CODE_DIR, 'main')],
    cwd=CODE_DIR, capture_output=True, text=True
)
elapsed = time.time() - t0

out = result.stdout
print(out[-4000:] if len(out) > 4000 else out)
if result.returncode != 0:
    print('STDERR:', result.stderr[-2000:])
    raise RuntimeError(f'Benchmark failed (exit {result.returncode})')

print(f'✓ Completed in {elapsed:.1f}s')

## Step 6 — Visualize

> **Why subprocess?**  
> The Intel oneAPI image ships a matplotlib compiled against an older numpy.
> Even after `pip install`, the kernel's already-loaded C extension (`.so`) is stale.  
> Running the viz in a **fresh subprocess** guarantees the updated binaries are used,  
> then the saved PNGs are displayed back here via `IPython.display`.

In [ ]:
import glob

def find_output(code_dir):
    patterns = [
        os.path.join(code_dir, 'Results_*', '**', 'Output_*'),
        os.path.join(code_dir, 'Results_*', '*'),
        os.path.join(code_dir, 'Output_*'),
    ]
    dirs = []
    for p in patterns:
        dirs += [d for d in glob.glob(p, recursive=True) if os.path.isdir(d)]
    return max(dirs, key=os.path.getmtime) if dirs else None

output_dir = find_output(CODE_DIR)
if output_dir:
    print(f'Output directory : {output_dir}')
    txts = sorted(f for f in os.listdir(output_dir) if f.endswith('.txt'))
    print(f'Files ({len(txts)}): {txts}')
else:
    raise RuntimeError('No output directory found — did Step 5 succeed?')

In [ ]:
import textwrap, tempfile
from IPython.display import Image, display

# Write the visualization script to a temp file and execute it in a fresh
# Python process so the force-reinstalled matplotlib is loaded from scratch.
VIZ_SCRIPT = textwrap.dedent(f'''
import os, sys, glob
import numpy as np
import matplotlib
matplotlib.use("Agg")   # non-interactive, works in subprocess
import matplotlib.pyplot as plt

output_dir = {repr(output_dir)}
BENCHMARK  = {repr(BENCHMARK)}
NK         = {NK}

j = lambda name: os.path.join(output_dir, name)

def load_meshgrid(path):
    if not os.path.exists(path): return None
    rows = []
    with open(path) as f:
        for line in f:
            if line.strip(): rows.append([float(v) for v in line.split()])
    return np.array(rows) if rows else None

def load_vector(path):
    if not os.path.exists(path): return None
    with open(path) as f:
        f.readline()
        rows = []
        for line in f:
            if line.strip():
                vals = [float(v) for v in line.split()]
                if vals: rows.append(vals)
    return np.array(rows) if rows else None

def load_points(path):
    if not os.path.exists(path): return None
    rows = []
    with open(path) as f:
        for line in f:
            if line.strip():
                vals = [float(v) for v in line.split()]
                if len(vals) >= 2: rows.append(vals)
    return np.array(rows) if rows else None

X_mesh = load_meshgrid(j("X_MeshGrid.txt"))
Y_mesh = load_meshgrid(j("Y_MeshGrid.txt"))
Y_proj = load_points(j("Y_projected.txt"))
_yp = load_points(j("Y_Pushed_projected.txt"))
Y_push = _yp if _yp is not None else load_points(j("Y_Pushed_projected_owndisc.txt"))
Ref    = load_vector(j("Ref_MY.txt"))
R_data = load_vector(j("R_MY.txt"))

# ── 6-panel main figure ───────────────────────────────────────────────────────
fig = plt.figure(figsize=(20, 13))
fig.suptitle(f"Reflector Benchmark — {{BENCHMARK}} / NK={{NK}}", fontsize=13, fontweight="bold")

def no_data(ax, msg):
    ax.text(0.5, 0.5, msg, ha="center", va="center", transform=ax.transAxes, color="grey")

ax = plt.subplot(2, 3, 1)
if X_mesh is not None:
    im = ax.imshow(X_mesh, extent=[-0.6,0.6,-0.6,0.6], origin="lower", cmap="viridis")
    plt.colorbar(im, ax=ax, label="Density")
    ax.set_title("Source Density", fontweight="bold")
else: no_data(ax, "X_MeshGrid.txt not found")
ax.set_xlabel("X"); ax.set_ylabel("Y")

ax = plt.subplot(2, 3, 2)
if Y_mesh is not None:
    im = ax.imshow(Y_mesh, extent=[-0.6,0.6,-0.6,0.6], origin="lower", cmap="plasma")
    plt.colorbar(im, ax=ax, label="Density")
    ax.set_title("Destination Density", fontweight="bold")
else: no_data(ax, "Y_MeshGrid.txt not found")
ax.set_xlabel("X"); ax.set_ylabel("Y")

ax = plt.subplot(2, 3, 3)
if Y_proj is not None and len(Y_proj):
    c = Y_proj[:,2] if Y_proj.shape[1] >= 3 else "blue"
    sc = ax.scatter(Y_proj[:,0], Y_proj[:,1], c=c, cmap="coolwarm", s=1, alpha=0.6)
    if Y_proj.shape[1] >= 3: plt.colorbar(sc, ax=ax, label="Z")
    ax.set_title("Target (original)", fontweight="bold")
    ax.set_aspect("equal"); ax.grid(True, alpha=0.3)
else: no_data(ax, "Y_projected.txt not found")
ax.set_xlabel("X"); ax.set_ylabel("Y")

ax = plt.subplot(2, 3, 4)
if Y_push is not None and len(Y_push):
    c = Y_push[:,2] if Y_push.shape[1] >= 3 else "red"
    sc = ax.scatter(Y_push[:,0], Y_push[:,1], c=c, cmap="coolwarm", s=1, alpha=0.6)
    if Y_push.shape[1] >= 3: plt.colorbar(sc, ax=ax, label="Z")
    ax.set_title("Pushed (reflected)", fontweight="bold")
    ax.set_aspect("equal"); ax.grid(True, alpha=0.3)
else: no_data(ax, "Y_Pushed_projected.txt not found")
ax.set_xlabel("X"); ax.set_ylabel("Y")

try:
    from mpl_toolkits.mplot3d import Axes3D
    _has_3d = True
except Exception:
    _has_3d = False
if _has_3d:
    ax = plt.subplot(2, 3, 5, projection="3d")
    if Ref is not None and len(Ref) and Ref.shape[1] >= 3:
        idx = np.random.choice(len(Ref), min(len(Ref), 10000), replace=False)
        ax.scatter(Ref[idx,0], Ref[idx,1], Ref[idx,2], s=0.5, alpha=0.6, c=Ref[idx,2], cmap="viridis")
        ax.set_title("Reflector Surface (3D)", fontweight="bold")
        ax.set_xlabel("X"); ax.set_ylabel("Y"); ax.set_zlabel("Z")
    else:
        ax.text2D(0.5, 0.5, "Ref_MY.txt not found", ha="center", va="center", transform=ax.transAxes)
else:
    ax = plt.subplot(2, 3, 5)
    if Ref is not None and len(Ref) and Ref.shape[1] >= 3:
        idx = np.random.choice(len(Ref), min(len(Ref), 10000), replace=False)
        sc = ax.scatter(Ref[idx,0], Ref[idx,1], c=Ref[idx,2], cmap="viridis", s=0.5, alpha=0.6)
        plt.colorbar(sc, ax=ax, label="Z")
        ax.set_title("Reflector Surface (2D, Z=colour)", fontweight="bold")
        ax.set_aspect("equal"); ax.grid(True, alpha=0.3)
    else:
        no_data(ax, "Ref_MY.txt not found")
ax.set_xlabel("X"); ax.set_ylabel("Y")

ax = plt.subplot(2, 3, 6)
if Y_proj is not None and Y_push is not None:
    ax.scatter(Y_proj[:,0], Y_proj[:,1], s=1, alpha=0.3, label="Target", color="steelblue")
    ax.scatter(Y_push[:,0], Y_push[:,1], s=1, alpha=0.3, label="Pushed", color="tomato")
    ax.legend(markerscale=5); ax.set_aspect("equal"); ax.grid(True, alpha=0.3)
    ax.set_title("Target vs Pushed", fontweight="bold")
else: no_data(ax, "comparison data unavailable")
ax.set_xlabel("X"); ax.set_ylabel("Y")

plt.tight_layout()
plt.savefig(j("visualization.png"), dpi=150, bbox_inches="tight")
print("SAVED:", j("visualization.png"))

# ── Radius analysis ───────────────────────────────────────────────────────────
if R_data is not None and Ref is not None:
    R_flat = R_data.flatten()
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle("Reflector Radius Analysis", fontweight="bold")
    axes[0].hist(R_flat, bins=60, edgecolor="black", alpha=0.7, color="skyblue")
    mean_r = np.mean(R_flat)
    axes[0].axvline(mean_r, color="red", ls="--", lw=2, label=f"Mean = {{mean_r:.4f}}")
    axes[0].legend(); axes[0].set_title("Radius Distribution")
    axes[0].set_xlabel("Radius"); axes[0].set_ylabel("Frequency")
    axes[0].grid(True, alpha=0.3)
    if Ref.shape[1] >= 3:
        idx = np.random.choice(len(Ref), min(len(Ref), 10000), replace=False)
        sc = axes[1].scatter(Ref[idx,0], Ref[idx,1], c=R_flat[idx], cmap="hot", s=1, alpha=0.8)
        plt.colorbar(sc, ax=axes[1], label="Radius")
        axes[1].set_aspect("equal"); axes[1].grid(True, alpha=0.3)
    axes[1].set_title("Reflector coloured by radius")
    axes[1].set_xlabel("X"); axes[1].set_ylabel("Y")
    plt.tight_layout()
    plt.savefig(j("reflector_analysis.png"), dpi=150, bbox_inches="tight")
    print("SAVED:", j("reflector_analysis.png"))

# ── Density comparison ────────────────────────────────────────────────────────
if X_mesh is not None and Y_mesh is not None:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle("Source vs Destination Density", fontweight="bold")
    im1 = axes[0].imshow(X_mesh, extent=[-0.6,0.6,-0.6,0.6], origin="lower", cmap="viridis")
    plt.colorbar(im1, ax=axes[0], label="Density")
    axes[0].set_title("Source"); axes[0].set_xlabel("X"); axes[0].set_ylabel("Y")
    im2 = axes[1].imshow(Y_mesh, extent=[-0.6,0.6,-0.6,0.6], origin="lower", cmap="plasma")
    plt.colorbar(im2, ax=axes[1], label="Density")
    axes[1].set_title("Destination"); axes[1].set_xlabel("X"); axes[1].set_ylabel("Y")
    plt.tight_layout()
    plt.savefig(j("density_comparison.png"), dpi=150, bbox_inches="tight")
    print("SAVED:", j("density_comparison.png"))
''')

# Write the script to a temp file next to the output so relative paths work
script_path = os.path.join(output_dir, '_viz.py')
with open(script_path, 'w') as f:
    f.write(VIZ_SCRIPT)

# Run in a fresh subprocess — the force-reinstalled matplotlib loads cleanly here
print('Running visualization subprocess...')
result = subprocess.run(
    [sys.executable, script_path],
    capture_output=True, text=True
)
print(result.stdout.strip())
if result.returncode != 0:
    print('STDERR:', result.stderr[-3000:])
    raise RuntimeError('Visualization subprocess failed')

# Display the saved PNGs inline
pngs = [
    os.path.join(output_dir, 'visualization.png'),
    os.path.join(output_dir, 'reflector_analysis.png'),
    os.path.join(output_dir, 'density_comparison.png'),
]
for png in pngs:
    if os.path.exists(png):
        print(f'\n── {os.path.basename(png)} ──')
        display(Image(png))